In [ ]:
import os
import datetime
import xlsxwriter
import win32com.client
import paramiko
import getpass
UID=input("Enter Userid :")
PWD=getpass.getpass("Enter password :")

#connecting to server
ssh = paramiko.SSHClient() 
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect("10.11.45.9", username=UID, password=PWD)
d={"Trend":"OLDW_Trend_Analysis_Fact_Tables_","NHI_vs_OLDW":"NHI_vs_OLDW_"}
src_path = r"C:\Users\\"+UID+"\Downloads\\"
today = datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)

last_mmyy=last_month.strftime("%b_%Y")
cur_mmyy=first.strftime("%b_%Y")

search_str= str(last_mmyy)
mmyy=first.strftime("%Y_%m")

print(cur_mmyy)

In [ ]:
#executing scripts
print("Executing script")
stdin,stdout,stderr=ssh.exec_command("sh /OLABS_QA/prod/NHI_OLDW_SCRIPT.sh")
out1=stderr.readlines()
stdin,stdout,stderr=ssh.exec_command("ls /OLABS_QA/prod/log/NHI_OLDW_fail.txt")
out2=stdout.readlines()
if(len(out2)==0):
    print("NHI_OLDW_SCRIPT.sh executed Successfully")
else:
    print("NHI_OLDW_SCRIPT.sh failed. Please fix the issue")
    print(out1)
    
#stdin,stdout,stderr=ssh_client.exec_command("sh /OLABS_QA/scripts/OPTLABTestAuto_Clean.sh NHI mnthly All_Sources_Live_vs_Nonlive_Base_percnt.sql")
#out=stdout.readlines()

In [ ]:
#find logs for current month
print("Please verify the below files which are being processed for metrics documents")
#d2={"All_Sources":["/OLABS_QA/prod/NHI/mnthly/output/SQL_All_Sources_Live_vs_Nonlive_Base_percnt_",""],"NHI_OLDW_BASE":["/OLABS_QA/prod/log/NHI_OLDW_BASE_DIST_",""]}
d2={"NHI_OLDW_BASE":["/OLABS_QA/prod/log/NHI_OLDW_BASE_DIST_",""]}
for key,value in d2.items():
    try:
        stdin,stdout,stderr=ssh.exec_command("ls -t {}{}* |head -1".format(value[0],mmyy))
        out=stdout.readlines()
        file=out[0].strip()
        value[1]=file
        print(file)
    except:
        print("Error finding file for "+key)
        print(stderr.readlines())
        
#converting numbers and decimals in logs from strings to their respective data types
def clean_num(line):
    for j in range(len(line)):
        line[j]=line[j].strip()
        if(line[j][0]=="-"):
            if(line[j][1:].isnumeric()):
                line[j]=0-int(line[j][1:])
            elif(line[j][1:].replace(".","").isnumeric()):
                line[j]=0.0-float(line[j][1:])
        else:
            if(line[j].isnumeric()):
                line[j]=int(line[j])
            elif(line[j].replace(".","").isnumeric()):
                line[j]=float(line[j])
    return line

#getting data from logs to temp excel file
with xlsxwriter.Workbook("C:\\Users\\"+UID+"\\Downloads\\temp_data.xlsx") as workbook:
    worksheet = workbook.add_worksheet(name="Trend")
    try:
        stdin,stdout,stderr=ssh.exec_command("ls -t /OLABS_QA/prod/NHI/mnthly/output/SQL_NHI_count_trend_mdfid_direct_{}*.log|head -1".format(mmyy))
        out=stdout.readlines()
        file=out[0].strip()
        print("File being used for counts : ",file)
    except:
        print("NHI_count_trend log file not found. Please run the current_vs_previous report file before Trend metrics report")
    stdin,stdout,stderr=ssh.exec_command("sh /OLABS_QA/scripts/Live_vs_Nonlive_Count_for_trend_metrics.sh {}".format(file))
    out=stderr.readlines()
    if(len(out)==0):
        print("Live_vs_Nonlive_Count_for_trend_metrics.sh script successful")
    else:
        print("Live_vs_Nonlive_Count_for_trend_metrics.sh execution failed")
        print(out)
    stdin,stdout,stderr=ssh.exec_command("cat /OLABS_QA/prod/log/Live_vs_Nonlive_Counts.txt")
    out=stdout.readlines()
    for i in range(len(out)):
        line=out[i].split("|")
        line=clean_num(line)
        worksheet.write_row(i,0,line)
    worksheet = workbook.add_worksheet(name="NHI_vs_OLDW")
    stdin,stdout,stderr=ssh.exec_command("cat {}".format(d2["NHI_OLDW_BASE"][1]))
    out=stdout.readlines()
    for i in range(len(out)-1):
        line=out[i].split("|")
        worksheet.write_row(i,0,line)


#Finding previous month file in server
def main_code(folder):
    stdin,stdout,stderr=ssh.exec_command("ls /OLABS_QA/prod/NHI/mnthly/Metrics_Files/{} | grep {}|wc -l".format(folder,search_str))
    output=stdout.readlines()
    if(output[0].strip()=="1"):
        print("Last month file found. Getting file to local")
        getfile(folder)
    else:
        print("File for previous month not found")

#getting last month file to local
def getfile(folder):
    remotefile="/OLABS_QA/prod/NHI/mnthly/Metrics_Files/"+folder+"/"+d[folder]+last_mmyy+".xlsm"
    last_mon_file=d[folder]+last_mmyy+".xlsm"
    cur_mon_file=d[folder]+cur_mmyy+".xlsm"
    final_file= src_path+"{}".format(cur_mon_file)
    sftp = ssh.open_sftp()
    try:
        sftp.get(remotefile,src_path+cur_mon_file)
        sftp.close()
        print("Last month file saved in local")
        file_processing(folder,final_file,cur_mon_file)
    except Exception as e:
        print("Getting last month file to Local failed")
        print(e)
        print(remotefile)
        print(src_path+cur_mon_file)

#running macros in excel file
def file_processing(folder,final_file,cur_mon_file):
    print("Processing ",final_file)
    oldw_final_file="/OLABS_QA/prod/NHI/mnthly/Metrics_Files/"+folder+"/"+cur_mon_file
    xl=win32com.client.Dispatch("Excel.Application")
    wb1=xl.Workbooks.Open(Filename="C:\\Users\\"+UID+"\\Downloads\\temp_data.xlsx")
    wb2=xl.Workbooks.Open(Filename=final_file)
    try:
        xl.Visible=True
        xl.Application.Run(cur_mon_file+"!Module1.macro1",cur_mon_file)
        wb1.Close()
        wb2.Close(SaveChanges=1)
        print("Success! file : "+final_file)
        transfer(oldw_final_file,final_file,folder)
    except:
        print("Macro processing failed")
        xl.Workbooks(1).Close(SaveChanges=0)
        xl.Workbooks(2).Close(SaveChanges=0)
        xl.Application.Quit()
    xl.Quit

#Transferring excel file to OLDW
def transfer(oldw_final_file,final_file,folder):
    sftp = ssh.open_sftp()
    sftp.put(final_file,oldw_final_file)
    sftp.close()
    print("File transferred successfully to OLABS server")
    
    send_mail(oldw_final_file,folder)

#Sending file over mail
def send_mail(oldw_final_file,folder):
    if(folder=="Trend"):
        sub="OLDW_Trend_Analysis "
    else:
        sub="NHI_vs_OLDW "
    stdin,stdout,stderr=ssh.exec_command("sh /OLABS_QA/scripts/SOPmetricsmail.sh {} {}".format(sub,oldw_final_file))
    out=stderr.readlines()
    if(len(out)==0):
        print("Report sent over mail successfully")
    else:
        print("Report mail failed")
        print(out)

# NHI_vs_OLDW Report
main_code("NHI_vs_OLDW")

# OLDW_Trend_Analysis Report
main_code("Trend")